<a href="https://colab.research.google.com/github/benny-png/CREATING-A-SPEECH-RECOGNITION-MODEL-ENDPOINT/blob/main/ASR_Model(no%20endpoint).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from huggingface_hub import notebook_login

notebook_login()

In [ ]:

"""
# import high-level pipeline

from transformers import pipeline

# This line will load your desired model
pipe = pipeline("automatic-speech-recognition", model="Akashpb13/Swahili_xlsr")





from transformers import AutoModelForASR, AutoTokenizer
# Replace 'username/model-name' with your Hugging Face username and model name
model_name = 'Akashpb13/Swahili_xlsr'

# Download the model and tokenizer
model = AutoModelForASR.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save the model and tokenizer to a directory
model_directory = "asr_model"
model.save_pretrained(model_directory)
tokenizer.save_pretrained(model_directory)

Akashpb13/Swahili_xlsr



from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Replace 'facebook/wav2vec2-base-960h' with the model identifier you want to download
model_name = 'Akashpb13/Swahili_xlsr'

# Download the model and tokenizer
model = Wav2Vec2ForCTC.from_pretrained(model_name)
tokenizer = Wav2Vec2Processor.from_pretrained(model_name)

# Save the model and tokenizer to a directory
model_directory = "asr_model2"
model.save_pretrained(model_directory)
tokenizer.save_pretrained(model_directory)




"""
# import high-level pipeline

from transformers import pipeline

# This line will load your desired model
pipe = pipeline("automatic-speech-recognition", model="Akashpb13/Swahili_xlsr")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at Akashpb13/Swahili_xlsr were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Akashpb13/Swahili_xlsr and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-

tokenizer_config.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [ ]:
!pip install torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install ffmpeg-python


In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
# to get an output, the file can be mp4, wav, or others

import os
path_audio="/content"
for filename in os.listdir(path_audio):
    if filename.endswith('.mp3'):
        # Create the full path to the audio file
        audio_path = os.path.join(path_audio, filename)
        print(pipe(audio_path))

# will get a transcribed output as a text

In [ ]:
audio, sr = get_audio()

KeyboardInterrupt: 

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)
# to get an output, the file can be mp4, wav, or others
print(pipe("recording.wav"))
# will get a transcribed output as a text

NameError: name 'sr' is not defined

In [ ]:
print(pipe("recording.wav"))

In [ ]:
audio_link = "https://17ba-41-86-177-7.ngrok-free.app/recording.wav"
print(pipe(f"{audio_link}"))

In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok

In [ ]:
# This downloads and sets up the Ngrok executable in the Google Colab instance
# Import the ngrok GPG key
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | gpg --import -

# Add the ngrok repository to the apt sources list
!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list

# Fetch the public key associated with the ngrok repository
!sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 0E61D3BBAAEE37FE

# Update the apt package lists
!sudo apt-get update

# Install ngrok
!sudo apt-get install ngrok


gpg: directory '/root/.gnupg' created
gpg: keybox '/root/.gnupg/pubring.kbx' created
gpg: /root/.gnupg/trustdb.gpg: trustdb created
gpg: key 0E61D3BBAAEE37FE: public key "ngrok agent apt repo release bot <release-bot@ngrok.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1
deb https://ngrok-agent.s3.amazonaws.com buster main
Executing: /tmp/apt-key-gpghome.yFErx0cnw9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 0E61D3BBAAEE37FE
gpg: key 0E61D3BBAAEE37FE: public key "ngrok agent apt repo release bot <release-bot@ngrok.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:4 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [4,555 B]
Hit:5 https://ppa.launchpadcontent.net

In [ ]:

# https://dashboard.ngrok.com/signup
!ngrok authtoken 2TdyYTihhC9d5wpYoJWC3DCoC9w_5LSdqY1oTkxQaGwsjbCHu

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import uvicorn
import torch

# Apply nest_asyncio
nest_asyncio.apply()

app = FastAPI()



class InferenceRequest(BaseModel):
    filename: str


@app.post("/inference/")
def inference(request: InferenceRequest):
    response = pipe(filename)
    return {"response": response}

# Start ngrok to tunnel the localhost
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the application
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [25624]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://a2c9-104-198-225-111.ngrok-free.app" -> "http://localhost:8000"
INFO:     34.74.97.97:0 - "POST / HTTP/1.1" 404 Not Found
INFO:     34.74.97.97:0 - "POST /inference HTTP/1.1" 307 Temporary Redirect
INFO:     34.74.97.97:0 - "POST /inference/ HTTP/1.1" 422 Unprocessable Entity
INFO:     41.86.177.7:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     41.86.177.7:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     41.86.177.7:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     41.86.177.7:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     34.74.97.97:0 - "POST /inference HTTP/1.1" 307 Temporary Redirect
INFO:     34.74.97.97:0 - "POST /inference/ HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25624]


KeyboardInterrupt: 

In [ ]:


import requests

# Define the endpoint URL
#url = "http://127.0.0.1:8000/gpt2-inference/"  # Replace with the ngrok URL if using ngrok

# Read the WAV file as binary data
with open("recording.wav", "rb") as file:
    audio_data = file.read()

# Send the POST request with the WAV file as the payload
response = requests.post(public_url, files={"file": ("audio.wav", audio_data)}, headers={"Content-Type": "audio/wav"})

# Print the response
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Failed to get response. Status code:", response.status_code)



In [ ]:
"""

import os
import torchaudio
import torch

# Specify the directory containing audio files
path_audio = "/content/Sample_audio"

# Iterate through audio files in the directory
for filename in os.listdir(path_audio):
    if filename.endswith(".mp3"):
        # Construct the full path to the audio file
        audio_file_path = os.path.join(path_audio, filename)

        # Load audio file
        waveform, sample_rate = torchaudio.load(audio_file_path)

        # Preprocess audio
        input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values

        # Perform inference
        with torch.no_grad():
            logits = model(input_values).logits

        # Decode predictions
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]

        print(f"Transcription for {filename}: {transcription}")
"""

'\n\nimport os\nimport torchaudio\nimport torch\n\n# Specify the directory containing audio files\npath_audio = "/content/Sample_audio"\n\n# Iterate through audio files in the directory\nfor filename in os.listdir(path_audio):\n    if filename.endswith(".mp3"):\n        # Construct the full path to the audio file\n        audio_file_path = os.path.join(path_audio, filename)\n\n        # Load audio file\n        waveform, sample_rate = torchaudio.load(audio_file_path)\n\n        # Preprocess audio\n        input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values\n\n        # Perform inference\n        with torch.no_grad():\n            logits = model(input_values).logits\n\n        # Decode predictions\n        predicted_ids = torch.argmax(logits, dim=-1)\n        transcription = processor.batch_decode(predicted_ids)[0]\n\n        print(f"Transcription for {filename}: {transcription}")\n'

In [ ]:
# to get an output, the file can be mp4, wav, or others



import os
path_audio="/content"
for filename in os.listdir(path_audio):
    if filename.endswith('.mp3'):
        # Create the full path to the audio file
        audio_path = os.path.join(path_audio, filename)
        print(pipe(audio_path))

# will get a transcribed output as a text

{'text': 'tangu kale anaheshimiwa sana na wakristo wa madhehebu mengi kama mtakatifu'}
{'text': 'dini yao ni uislamu'}
{'text': 'inaitwa si kwa sababu ni mageuzi ya b lugha ya programu nyingine'}
{'text': 'aliyemfuata kama tenno ni mwana wake gouda'}
{'text': 'kamusi hiyo inapatikana kwa njia ya tovuti yake'}
{'text': 'parokia hii ipo katika wilaya ya bukoba vijijini mkoani kagera kaskazini magharibi mwa tanzania'}
{'text': 'mpaka hileo huduma hii inaendelea'}
{'text': 'ni maarufu sana kwa nchini humo na nje ya nchi ya utoruki'}
{'text': 'aina hizi hutegemea matumizi na mabadiliko yake kwa mfano'}
{'text': 'piko mjini mombasa katika pwani ya kenya'}
{'text': 'hapo aliweka anadhiri za daima kwa jina la maria wa yesu katika sakramenti'}


In [ ]:
audio, sr = get_audio()

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)
# to get an output, the file can be mp4, wav, or others
print(pipe("recording.wav"))
# will get a transcribed output as a text

{'text': 'kama kuna mtu anaweza kuamka asubuhi aniamishei'}


In [ ]:
https://b98c-34-82-209-94.ngrok-free.app

In [ ]:
https://b98c-34-82-209-94.ngrok-free.app

# New Section